<a href="https://colab.research.google.com/github/Hao0827/2-1-EATLAND/blob/main/%EB%95%85%EB%94%B0%EB%A8%B9%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pygame
import sys
import math
import random

WINDOW_WIDTH = 1260
WINDOW_HEIGHT = 720
FPS = 30

# Colors
BACKGROUND = (30, 30, 30)
WHITE = (255, 255, 255)
BUTTON_COLOR = (70, 70, 70)
BUTTON_HOVER = (100, 100, 100)
TEXT_COLOR = WHITE
GRID_LINE_COLOR = (50, 50, 50)
UNCLAIMED_COLOR = (200, 200, 200)
OVERLAY_COLOR = (0, 0, 0)

# Team colors
TEAM_COLORS = {1: (255, 100, 100), 2: (100, 100, 255)}

# Hex Config
HEX_RADIUS = 2  # 큰 정육각형 한 변에 작은 육각형 4개가 있게 하기 위해 반지름을 2로 설정
HEX_SIZE = 40

# Habitat types and colors (unclaimed display)
habitats = {
    'forest':    (34, 139, 34),
    'grassland': (124, 252, 0),
    'desert':    (210, 180, 140),
    'wetland':   (70, 130, 180),
    'wasteland': (105, 105, 105),
    'river':     (0, 191, 255),
    'ocean':     (0, 0, 128),
    'mudflat':   (160, 82, 45),
    'cave':      (47, 79, 79),
    'polar':     (224, 255, 255),
    'simhae':    (0, 0, 64)
}
# Korean names for display
habitat_names = {
    'forest':    '숲',
    'grassland': '초원',
    'desert':    '사막',
    'wetland':   '습지',
    'wasteland': '황무지',
    'river':     '강',
    'pond':      '연못',
    'ocean':     '바다',
    'mudflat':   '갯벌',
    'cave':      '동굴',
    'polar':     '극지방',
    'simhae':    '심해'
}

# Easy habitats around wasteland
EASY_HABITATS = ['grassland', 'pond', 'river', 'forest', 'ocean']

# Neighbor deltas
NEIGHBORS = [(1, 0), (0, 1), (-1, 1), (-1, 0), (0, -1), (1, -1)]

# Initialize pygame
pygame.init()
screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("서식지 땅따먹기 게임")
clock = pygame.time.Clock()

# Font
try:
    title_font = pygame.font.Font('온글잎 콘콘체.ttf', 60)
    button_font = pygame.font.Font('온글잎 콘콘체.ttf', 50)
    small_font = pygame.font.Font('온글잎 콘콘체.ttf', 35)
except:
    title_font = pygame.font.Font(None, 48)
    button_font = pygame.font.Font(None, 32)
    small_font = pygame.font.Font(None, 24)

# Game state
state = 'menu'
current_team = 1
selection_mode = False
selected_hex = None

# Create grid
def hex_corners(cx, cy, size):
    return [(cx + size * math.cos(math.radians(60 * i)),
             cy + size * math.sin(math.radians(60 * i))) for i in range(6)]

hex_centers = []
for q in range(-HEX_RADIUS, HEX_RADIUS + 1):
    for r in range(-HEX_RADIUS, HEX_RADIUS + 1):
        if abs(-q - r) <= HEX_RADIUS:
            x = HEX_SIZE * (3/2 * q)
            y = HEX_SIZE * (math.sqrt(3) * (r + q/2))
            hex_centers.append({'q': q, 'r': r, 'center': (WINDOW_WIDTH//2 + x, WINDOW_HEIGHT//2 + y)})

# Assign habitats
# 1) 무작위로 황무지 3개 선택
waste = random.sample(hex_centers, 3)
for h in waste:
    h['habitat'] = 'wasteland'
    h['owner'] = -1
# 2) 황무지 주변에 EASY_HABITATS 중복 최소화하며 배치
for w in waste:
    # 인접 가능한 후보
    neighbors = [u for u in hex_centers if abs(u['q'] - w['q']) + abs(u['r'] - w['r']) == 1 or (u['q'] - w['q'] == 1 and u['r'] - w['r'] == -1) or (u['q'] - w['q'] == -1 and u['r'] - w['r'] == 1)]
    # 활용 가능한 EASY_HABITATS 복사본 생성
    choices = EASY_HABITATS.copy()
    random.shuffle(choices)
    for idx, h in enumerate(neighbors):
        if 'habitat' in h:
            continue
        # 순차적으로 EASY_HABITATS 할당, 부족 시 다시 셔플
        if idx >= len(choices):
            choices = EASY_HABITATS.copy()
            random.shuffle(choices)
            idx = 0
        h['habitat'] = choices[idx]
        h['owner'] = 0
# 3) 남은 칸에 남은 서식지를 중복 최소화하여 배치
remain_types = [k for k in habitats.keys() if k != 'wasteland' and k not in EASY_HABITATS]
unassigned = [h for h in hex_centers if 'habitat' not in h]
# 필요한 수만큼 remain_types 반복
count_unassigned = len(unassigned)
import math
repeats = math.ceil(count_unassigned / len(remain_types))
pool = remain_types * repeats
random.shuffle(pool)
for i, h in enumerate(unassigned):
    h['habitat'] = pool[i]
    h['owner'] = 0

# Helpers
def point_in_poly(x, y, poly):
    inside = False
    j = len(poly) - 1
    for i in range(len(poly)):
        xi, yi = poly[i]
        xj, yj = poly[j]
        if ((yi > y) != (yj > y)) and (x < (xj - xi) * (y - yi) / (yj - yi) + xi):
            inside = not inside
        j = i
    return inside

def count_owned_neighbors(q, r, grid, owner):
    cnt = 0
    for dq, dr in NEIGHBORS:
        for h in grid:
            if h['q'] == q + dq and h['r'] == r + dr and h['owner'] == owner:
                cnt += 1
                break
    return cnt

def draw_button(surface, rect, text, font, hover=False):
    color = BUTTON_HOVER if hover else BUTTON_COLOR
    pygame.draw.rect(surface, color, rect)
    txt = font.render(text, True, TEXT_COLOR)
    surface.blit(txt, (rect.x + (rect.width - txt.get_width()) // 2,
                       rect.y + (rect.height - txt.get_height()) // 2))

# UI rects
start_button = pygame.Rect((WINDOW_WIDTH//2 - 150, WINDOW_HEIGHT//2 - 40), (300, 80))
back_button  = pygame.Rect(10, 10, 100, 40)
s_button     = pygame.Rect(WINDOW_WIDTH//2 - 160, WINDOW_HEIGHT - 100, 140, 50)
f_button     = pygame.Rect(WINDOW_WIDTH//2 + 20, WINDOW_HEIGHT - 100, 140, 50)

# Main loop
running = True
while running:
    mx, my = pygame.mouse.get_pos()
    click = False
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            click = True
            if state == 'menu' and start_button.collidepoint(mx, my):
                state = 'hex'
            elif state == 'hex':
                if back_button.collidepoint(mx, my):
                    state = 'menu'
                    selection_mode = False
                    selected_hex = None
                elif not selection_mode:
                    for h in hex_centers:
                        poly = hex_corners(*h['center'], HEX_SIZE)
                        if point_in_poly(mx, my, poly) and h['owner'] == 0:
                            selected_hex = h
                            selection_mode = True
                            break
                else:
                    if s_button.collidepoint(mx, my):
                        selected_hex['owner'] = current_team
                    if s_button.collidepoint(mx, my) or f_button.collidepoint(mx, my):
                        current_team = 2 if current_team == 1 else 1
                        selection_mode = False
                        selected_hex = None

    # Draw
    screen.fill(BACKGROUND)
    if state == 'menu':
        title = title_font.render("서식지 땅따먹기", True, TEXT_COLOR)
        screen.blit(title, ((WINDOW_WIDTH - title.get_width()) // 2, 80))
        hover = start_button.collidepoint(mx, my)
        draw_button(screen, start_button, "게임 시작", button_font, hover)
    else:
        hover_back = back_button.collidepoint(mx, my)
        draw_button(screen, back_button, "뒤로", small_font, hover_back)
        turn_txt = small_font.render(f"팀 {current_team} 턴", True, TEXT_COLOR)
        screen.blit(turn_txt, (WINDOW_WIDTH - 120, 10))
        red_score = blue_score = 0
        for h in hex_centers:
            if h['owner'] in TEAM_COLORS:
                fill = TEAM_COLORS[h['owner']]
            elif h['habitat'] == 'wasteland':
                fill = habitats['wasteland']
            else:
                fill = UNCLAIMED_COLOR
            pygame.draw.polygon(screen, fill, hex_corners(*h['center'], HEX_SIZE))
            pygame.draw.polygon(screen, GRID_LINE_COLOR, hex_corners(*h['center'], HEX_SIZE), 2)
            if h['owner'] in TEAM_COLORS:
                n = count_owned_neighbors(h['q'], h['r'], hex_centers, h['owner'])
                pts = n / 2
                if h['owner'] == 1:
                    red_score += pts
                else:
                    blue_score += pts
        score_txt = small_font.render(f"Red: {int(red_score)}  Blue: {int(blue_score)}", True, WHITE)
        screen.blit(score_txt, (10, WINDOW_HEIGHT - 50))
        if selection_mode and selected_hex:
            pygame.draw.rect(screen, OVERLAY_COLOR, (0, WINDOW_HEIGHT - 200, WINDOW_WIDTH, 200))
            name = habitat_names[selected_hex['habitat']]
            lbl = small_font.render(name, True, WHITE)
            screen.blit(lbl, ((WINDOW_WIDTH - lbl.get_width()) // 2, WINDOW_HEIGHT - 170))
            hover_s = s_button.collidepoint(mx, my)
            hover_f = f_button.collidepoint(mx, my)
            draw_button(screen, s_button, "성공", small_font, hover_s)
            draw_button(screen, f_button, "실패", small_font, hover_f)

    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()
sys.exit()
